In [1]:
%cd

/home/gyubin


/home/gyubin/anaconda3/envs/EasyEdit/lib/python3.9/site-packages/IPython/core/magics/osm.py:393: UserWarning: using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})
/home/gyubin/anaconda3/envs/EasyEdit/lib/python3.9/site-packages/IPython/core/magics/osm.py:428: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [ ]:
import os
import openai
import os

os.environ['OPENAI_API_KEY'] = "MY_API_KEY"

print(os.getenv('OPENAI_API_KEY'))
openai.api_key = os.getenv("OPENAI_API_KEY")


In [ ]:
# should check the version

# openai.api_key = os.getenv("OPENAI_API_KEY")

# from openai import OpenAI

# client = OpenAI()


In [ ]:
import pandas as pd
sbj_path = './Knowledge_Editing_Dataset/df_one_hop_sbj_34.xlsx'
obj_true_path = './Knowledge_Editing_Dataset/df_one_hop_obj_true_34.xlsx'
obj_new_path = './Knowledge_Editing_Dataset/df_one_hop_obj_new_34.xlsx'
df1 = pd.read_excel(obj_true_path)
df1.head()
word = df1.loc[0,'one_hop']
word = ''
#if pd.isna(word):
 #   print(f'hello {word}')
print(f'hello {word}')

In [11]:
import openai
import json
import pandas as pd
from tqdm import tqdm

# 프롬프트 생성 함수
def generate_prompt(edit_sentence, subject, obj_true, obj_new, one_hop):
    return f"""
    Create 5 sentences according to the rules below.

    Rules:
    1. Each sentence must be naturally coherent when placed before the following sentence: "{edit_sentence}".
    2. The following 3 words must *never* be included in the sentences: "3 words: {subject}, {obj_true}, {obj_new}".
    3. If the following word list is not empty, at least one word from the list must be included in each sentence: "words list: {one_hop}".
    4. The words specified in rule 3 must be selected evenly, without bias.
    5. Each new sentence must consist of only one sentence.
    6. In total, 5 sentences must be generated.

    The output should always follow the JSON format, where 'sentence' is the key.

    Example format:

    {{
        "sentence" : [
            {{
                "one": "The first sentence you generated",
                "two": "The second sentence you generated",
                "three": "The third sentence you generated",
                "four": "The fourth sentence you generated",
                "five": "The fifth sentence you generated"
            }}
        ]
    }}
    """

def generate_prompts(data_path, sbj_path, obj_true_path, obj_new_path, model="gpt-4o-mini", max_tokens=1000):

    # 엑셀 파일 및 JSON 파일 읽기
    sbj_data = pd.read_excel(sbj_path)
    obj_true_data = pd.read_excel(obj_true_path)
    obj_new_data = pd.read_excel(obj_new_path)

    with open(data_path, 'r') as f:
        data = json.load(f)

    results = []

    # OpenAI 클라이언트 초기화 (API 키 설정 필요)
    client = openai

    for i in tqdm(range(len(data))):
        try:
            edit_sentence = data[i]['prompts'][0].format(data[i]['subject']) + " " + data[i]['fact_knowledge']
            subject = data[i]['subject']
            obj_true = data[i]['fact_knowledge']
            obj_new = data[i]['edited_knowledge']

            sbj_one_hop = sbj_data.loc[i, 'one_hop'] if not pd.isna(sbj_data.loc[i, 'one_hop']) else ''
            obj_true_one_hop = obj_true_data.loc[i, 'one_hop'] if not pd.isna(obj_true_data.loc[i, 'one_hop']) else ''
            obj_new_one_hop = obj_new_data.loc[i, 'one_hop'] if not pd.isna(obj_new_data.loc[i, 'one_hop']) else ''

            # 주제에 대한 프롬프트 생성
            prompt_sbj = generate_prompt(edit_sentence, subject, obj_true, obj_new, sbj_one_hop)
            prompt_obj_true = generate_prompt(edit_sentence, subject, obj_true, obj_new, obj_true_one_hop)
            prompt_obj_new = generate_prompt(edit_sentence, subject, obj_true, obj_new, obj_new_one_hop)

            # OpenAI API 호출 함수
            def call_openai_api(prompt):
                response = openai.ChatCompletion.create(
                    model=model,
                    messages=[
                        {"role": "system", "content": f"{prompt} Respond with a JSON object containing the generated sentences."},
                        {"role": "user", "content": "Generate a JSON containing the specified contextual sentences."}
                    ],
                    max_tokens=max_tokens
                )
                return json.loads(response.choices[0].message['content'])

            # 각각의 API 호출 결과 처리
            try:
                generated_content_sbj = call_openai_api(prompt_sbj)
                generated_content_obj_true = call_openai_api(prompt_obj_true)
                generated_content_obj_new = call_openai_api(prompt_obj_new)
            except Exception as e:
                print(f"Error generating content for item {i}: {e}")
                continue

            # 결과 저장
            results.append({
                "subject": subject,
                "fact_knowledge": obj_true,
                "edited_knowledge": obj_new,
                "relation_id": data[i]['relation_id'],
                "prompts": data[i]['prompts'],
                "sbj_one_hop": sbj_one_hop,
                "obj_true_one_hop": obj_true_one_hop,
                "obj_new_one_hop": obj_new_one_hop,
                "answers": {
                    "sbj": generated_content_sbj,
                    "obj_true": generated_content_obj_true,
                    "obj_new": generated_content_obj_new
                }
            })

        except Exception as e:
            print(f"Error occurred in processing item {i}: {e}")
            continue

    return json.dumps(results, indent=4)
#json_file_path = './gen_prompts_v3_gpt-4o.json'

sbj_path = './Knowledge_Editing_Dataset/df_one_hop_sbj_34.xlsx'
obj_true_path = './Knowledge_Editing_Dataset/df_one_hop_obj_true_34.xlsx'
obj_new_path = './Knowledge_Editing_Dataset/df_one_hop_obj_new_34.xlsx'
data_path = "./Knowledge_Editing_Dataset/data/one_hop/editcase-per-34-relations.json"
result = generate_prompts(data_path, sbj_path, obj_true_path, obj_new_path,  model="gpt-4o-mini", max_tokens=1000)

with open('hop_34_gpt-4o-mini_3.json', 'w') as f:
    f.write(result)

In [14]:
import openai
import json
import pandas as pd
from tqdm import tqdm

# 문맥 pointing

# 프롬프트 생성 함수
def generate_prompt(edit_sentence, subject, obj_true, obj_new, one_hop, target):
    return f"""
    Create 5 sentences according to the rules below.

    Rules:
    1. Each sentence must be naturally coherent when placed before the following sentence: "{edit_sentence}".
    2. The following 3 words must *never* be included in the sentences: "3 words: {subject}, {obj_true}, {obj_new}".
    3. If the following word list is not empty, at least one word from the list must be included in each sentence: "words list: {one_hop}".
    4. The words specified in rule 3 must be selected evenly, without bias.
    5. When generating contextual sentences, create them to fit the context of {target}.
    6. Each new sentence must consist of only one sentence.
    7. In total, 5 sentences must be generated.

    The output should always follow the JSON format, where 'sentence' is the key.

    Example format:

    {{
        "sentence" : [
            {{
                "one": "The first sentence you generated",
                "two": "The second sentence you generated",
                "three": "The third sentence you generated",
                "four": "The fourth sentence you generated",
                "five": "The fifth sentence you generated"
            }}
        ]
    }}
    """

def generate_prompts(data_path, sbj_path, obj_true_path, obj_new_path, model="gpt-4o-mini", max_tokens=1000):

    # 엑셀 파일 및 JSON 파일 읽기
    sbj_data = pd.read_excel(sbj_path)
    obj_true_data = pd.read_excel(obj_true_path)
    obj_new_data = pd.read_excel(obj_new_path)

    with open(data_path, 'r') as f:
        data = json.load(f)

    results = []

    # OpenAI 클라이언트 초기화 (API 키 설정 필요)
    client = openai

    for i in tqdm(range(len(data))):
        try:
            edit_sentence = data[i]['prompts'][0].format(data[i]['subject']) + " " + data[i]['fact_knowledge']
            subject = data[i]['subject']
            obj_true = data[i]['fact_knowledge']
            obj_new = data[i]['edited_knowledge']

            sbj_one_hop = sbj_data.loc[i, 'one_hop'] if not pd.isna(sbj_data.loc[i, 'one_hop']) else ''
            obj_true_one_hop = obj_true_data.loc[i, 'one_hop'] if not pd.isna(obj_true_data.loc[i, 'one_hop']) else ''
            obj_new_one_hop = obj_new_data.loc[i, 'one_hop'] if not pd.isna(obj_new_data.loc[i, 'one_hop']) else ''

            # 주제에 대한 프롬프트 생성
            prompt_sbj = generate_prompt(edit_sentence, subject, obj_true, obj_new, sbj_one_hop, target = subject)
            prompt_obj_true = generate_prompt(edit_sentence, subject, obj_true, obj_new, obj_true_one_hop, target = obj_true)
            prompt_obj_new = generate_prompt(edit_sentence, subject, obj_true, obj_new, obj_new_one_hop, target = obj_new)

            # OpenAI API 호출 함수
            def call_openai_api(prompt):
                response = openai.ChatCompletion.create(
                    model=model,
                    messages=[
                        {"role": "system", "content": f"{prompt} Respond with a JSON object containing the generated sentences."},
                        {"role": "user", "content": "Generate a JSON containing the specified contextual sentences."}
                    ],
                    max_tokens=max_tokens
                )
                return json.loads(response.choices[0].message['content'])

            # 각각의 API 호출 결과 처리
            try:
                generated_content_sbj = call_openai_api(prompt_sbj)
                generated_content_obj_true = call_openai_api(prompt_obj_true)
                generated_content_obj_new = call_openai_api(prompt_obj_new)
            except Exception as e:
                print(f"Error generating content for item {i}: {e}")
                continue

            # 결과 저장
            results.append({
                "subject": subject,
                "fact_knowledge": obj_true,
                "edited_knowledge": obj_new,
                "relation_id": data[i]['relation_id'],
                "prompts": data[i]['prompts'],
                "sbj_one_hop": sbj_one_hop,
                "obj_true_one_hop": obj_true_one_hop,
                "obj_new_one_hop": obj_new_one_hop,
                "answers": {
                    "sbj": generated_content_sbj,
                    "obj_true": generated_content_obj_true,
                    "obj_new": generated_content_obj_new
                }
            })

        except Exception as e:
            print(f"Error occurred in processing item {i}: {e}")
            continue

    return json.dumps(results, indent=4)
#json_file_path = './gen_prompts_v3_gpt-4o.json'

sbj_path = './Knowledge_Editing_Dataset/df_one_hop_sbj_34.xlsx'
obj_true_path = './Knowledge_Editing_Dataset/df_one_hop_obj_true_34.xlsx'
obj_new_path = './Knowledge_Editing_Dataset/df_one_hop_obj_new_34.xlsx'
data_path = "./Knowledge_Editing_Dataset/data/one_hop/editcase-per-34-relations.json"
result = generate_prompts(data_path, sbj_path, obj_true_path, obj_new_path,  model="gpt-4o-mini", max_tokens=1000)

with open('hop_34_gpt-4o-mini_4.json', 'w') as f:
    f.write(result)

100%|██████████| 34/34 [04:13<00:00,  7.45s/it]
